In [ ]:
%load_ext autoreload
%autoreload 2
import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
from scipy.optimize import nnls
from sklearn.decomposition import FastICA
from matplotlib.colors import ListedColormap
from sklearn.decomposition import NMF
%matplotlib inline
from utils import *
from vca import *
from model.nmf import NMFGD

In [ ]:
phantom_list = sorted([int(wave[:3]) for wave in os.listdir('../expdata/20230217 Cholesterol and Blood Mix/') if (wave[-3:] == 'mat' and wave[0] != 'S')])
abs_coeff, legend = {}, ['Blood Mixed', 'Cholesterol', 'Background']
wave_abs = np.load('./data/hbo2hbchpr_57.npy')

f = loadmat("./data/unmix.mat")
X, Y = f['x'], f['y']

for idx, wave in enumerate(np.arange(700, 981, 5)):
    abs_coeff[wave] = (idx, wave_abs[idx])
coeffs = np.vstack([abs_coeff[wave][1] for wave in phantom_list])

phantom_coeffs = np.vstack([abs_coeff[wave][1] for wave in phantom_list])
for i in range(phantom_coeffs.shape[0]):
    phantom_coeffs[i][1] = (phantom_coeffs[i][0] * 0.1) + (phantom_coeffs[i][1] * 0.9)
bloodmixch = phantom_coeffs[:,1:3]
weights_plot(array = bloodmixch, wave_list = phantom_list, legend = ['Blood Mix', 'Cholesterol'], figsize = (8, 6), xticks = phantom_list, title = '')

In [ ]:
phantom_data = np.array([np.array(loadmat(f"./data/blood_ch_phantom/PA_Image_{wave}.mat")['Image_PA']) for wave in phantom_list])
phantom_data_plot = normalize(phantom_data.copy())
plt.figure(figsize = (24, 16))
for i in range(18):
    plt.subplot(3, 6, i + 1)
    plt.imshow(phantom_data_plot[i], cmap = 'jet')
    plt.title(label = f'{phantom_list[i]}nm')
    plt.colorbar()
plt.show()
del phantom_data_plot

## Linear Unmixing

In [ ]:
phantom_linear = run_linear_unmixing(normalize(phantom_data.copy()), bloodmixch)
plot_comps_2d(phantom_linear, phantom_list, bloodmixch, clim = [None]*3, xticks = None, order = [0, 1])

In [ ]:
plot_3d_multiple(Y*1000, X*1000, phantom_linear, title = ['Blood Mix', 'Cholesterol', 'Background'], cmap = 'jet', clim = [None]*3, order = [0, 1, 2])

## ICA

In [ ]:
for i in range(20):
    print(f"Random State: {i}")
    maps, wts, _ = run_ica(phantom_data, phantom_list, 2, i)
    plot_comps_2d(maps, phantom_list, wts, figsize = (10, 3), order = [0, 1])

In [ ]:
maps, wts, model = run_ica(phantom_data, phantom_list, 2, 0)
plot_comps_2d(maps, phantom_list, np.linalg.pinv(model.components_), clim = [None]*3, xticks = None, order = [0, 1])

In [ ]:
plot_3d_multiple(Y*1000, X*1000, maps, title = ['Blood Mix', 'Cholesterol'], cmap = 'jet', clim = [None]*2, order = [1, 0])

## NMF

In [ ]:
nmf_model = NMF(n_components = 3)
nmf_model.fit(normalize(phantom_data.copy()).reshape(len(phantom_list), -1).T)
nmf_comps = nmf_model.fit_transform(normalize(phantom_data.copy()).reshape(len(phantom_list), -1).T).reshape((396, 101, 3))
plot_comps_2d(nmf_comps, phantom_list, nmf_model.components_.T, order = [0, 1], clim = [None]*3, chrom = ['Blood Mix', 'Cholesterol'], title = 'NMF')

## NMFGD

In [ ]:
nmf_model_test = NMFGD(n_components = 2, randominit = True)
nmf_model_test.fit(normalize(phantom_data.copy()).reshape((len(phantom_list), -1)), maxiter = 1500)

In [ ]:
plot_comps_2d(nmf_model_test.H.T.reshape((396, 101, 2)), phantom_list, nmf_model_test.W, order = [0, 1], clim = [None]*3)

## VCA

In [ ]:
ae, ind, yp = vca(normalize(phantom_data.copy()).reshape(len(phantom_list), -1), 2)
vca_comps = np.matmul(np.linalg.pinv(ae), yp).reshape((2, 396, 101)).transpose((1, 2, 0))
plot_comps_2d(vca_comps, phantom_list, ae, order = [0, 1], clim = [None]*2, chrom = ['Blood Mix', 'Cholesterol'], title = 'VCA')

## Phantom Cholesterol Experiments

```python
expdata = np.array([np.mean(np.array(loadmat(f'/content/20230305_Cholesterol_3/Phantom 1/{wave}nm.mat')['ImgData'])[1][0], axis = 3)[:,:,0] for wave in wave_list])
wave, h, w = expdata.shape
```

In [ ]:
os.listdir('./expdata/')

## Wavelength Data

In [ ]:
INDEX = 13
path = './expdata/20230329 Cholesterol and Blood Mix/'
wave_data = np.array(loadmat(f'{path}{phantom_list[INDEX]}nm.mat')['ImgData'][1][0])

print(f'{phantom_list[INDEX]}nm')
plt.figure(figsize = (30, 10))
for i in range(wave_data.shape[-1]):
    plt.subplot(2, 10, i+1)
    plt.imshow(wave_data[:,:,0,i], cmap = 'hot')
    plt.title(f'{np.std(wave_data[:,:,0,i]):.3f}')
    plt.colorbar()
plt.show()

In [ ]:
wavefiles = [wavefile for wavefile in os.listdir(path) if wavefile[-3:] == 'mat' and len(wavefile) == 9]
wavefiles = sorted(wavefiles, key = lambda filename: int(filename[:3]))
dim = loadmat(f'{path}/{wavefiles[0]}')['ImgData'][1][0].shape

In [ ]:
threshdict = {}
for idx, wave in enumerate(phantom_list):
    f = loadmat(f'{path}{wave}nm.mat')['ImgData'][1][0]
    for frame in range(f.shape[-1]):
        threshdict.setdefault(f'{wave}', []).append(np.std(f[:,:,0,frame]))

In [ ]:
THRESH = 3000
expdata, explist = np.empty((len(phantom_list), dim[0], dim[1])), []

for idx, wave in enumerate(phantom_list):
    wave_data = np.array(loadmat(f'{path}/{wave}nm.mat')['ImgData'][1][0])
    std_values = np.std(wave_data, axis = (0, 1, 2))
    wave_data_list = np.where(std_values > THRESH)[0]
    print(f'{idx}: {len(wave_data_list)}')
    wave_data_mean = np.mean(wave_data[:, :, 0, [wave_data_list]], axis = 3)
    expdata[idx, :, :] = wave_data_mean[:, :, 0]
    explist.append(len(wave_data_list))

In [ ]:
expdata_plot = normalize(expdata.copy())
plt.figure(figsize = (30, 12))
for i in range(len(phantom_list)):
    plt.subplot(2, 9, i + 1)
    plt.imshow(20 * np.log10(expdata_plot[i]), cmap = "hot")
    plt.title(label = f'{phantom_list[i]}nm')
    plt.colorbar()
    plt.clim([-60, 0])
plt.show()
del expdata_plot

In [ ]:
phantom_exp = run_linear_unmixing(normalize(expdata.copy()), bloodmixch)
plot_comps_2d(phantom_exp, phantom_list, bloodmixch, clim = [None]*3, xticks = None, order = [0, 1], chrom = ['Blood Mix', 'Cholesterol'])